TABU- HEURISTIC 1

In [1]:
import numpy as np
import random
import pandas as pd
import copy
from collections import defaultdict
from networkx import nx
from copy import copy
import time
import pickle

In [2]:
adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n300-m1200.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2


zeros=[1,0.3,0.1]
ones=[1,0.5,0.3]

In [3]:
adjacency_df.loc[adjacency_df.loc[:,range(1,n+1)].sum(axis=0)<5,:].index

Int64Index([  1,   2,   3,   4,   5,   6,   8,   9,  12,  15,
            ...
            290, 291, 292, 293, 294, 296, 297, 298, 299, 300],
           dtype='int64', length=225)

In [4]:
def convert(a):      
    rows, cols = np.where(a.loc[range(1,n+1), range(1,n+1)] > 0)
    edges = zip((rows+1).tolist(), (cols+1).tolist())
    G = nx.Graph()
    G.add_edges_from(edges)

    return G

In [5]:
#parameters
Blocking_Capacity = (int(n/10))*2
step_size=3

In [6]:
#sets

setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()


In [7]:
#Preprocessing
preprocessed_matrix = adjacency_df.copy(deep=True)



In [8]:
#2 infected 
preprocessed_matrix.loc[setP,setP] = 0




In [9]:
#Sum of all edges lower than threshold (not infected)
index = preprocessed_matrix.loc[(preprocessed_matrix[setV].sum(axis=1)<((preprocessed_matrix.loc[:, "Threshold"]))) & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)  , range(1,n+1)].index
preprocessed_matrix.loc[index,setV] = 0
preprocessed_matrix.loc[setV,index] = 0



In [10]:
#1 neighbor (not infected not risky)
index2 = preprocessed_matrix.loc[((preprocessed_matrix.loc[setV,setV]>0).sum(axis=1) == 1 & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)) & (preprocessed_matrix.loc[:,"IsRisky"]<0.01), range(1,n+1)].index
preprocessed_matrix.loc[index2,setV] = 0
preprocessed_matrix.loc[setV,index2] = 0

In [11]:
#Percentage assignment
preprocessed_matrix.loc[:,setV]=preprocessed_matrix.loc[:,setV].div(preprocessed_matrix.loc[:,"Threshold"],axis=0)
preprocessed_matrix[preprocessed_matrix[setV] >= 1] = 1


In [12]:
nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [13]:
#!!!!!!!!! Edges below %50 influence are deleted !!!!!!!!!!!
Graph3=convert(preprocessed_matrix)
preprocessed_matrix[preprocessed_matrix[setV] <= 0.3] = 0

In [14]:
nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [15]:
preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index

Int64Index([], dtype='int64')

In [16]:
Graph = convert(preprocessed_matrix)
#Graph.nodes()

In [17]:
found_paths=[]
for starting_nodes in setP:
    for ending_nodes in setRisky:
        found_paths=found_paths+list(nx.all_simple_paths(Graph, starting_nodes, ending_nodes, cutoff=step_size))  

In [18]:
for j in setP:
    found_paths = [[ele for ele in sub if ele != j] for sub in found_paths]

In [19]:
remaining_blocking_capacity=Blocking_Capacity
found_paths_removable=found_paths
already_blocked_nodes=[]
found_paths_np = np.concatenate([np.array(i) for i in found_paths])

In [20]:
nodes_on_paths, total_appearance = np.unique(found_paths_np, return_counts=True)

for i in setP:
    total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))
#for i in setRisky:
    #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
    #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))    


In [21]:
#nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]



In [22]:
already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
remaining_blocking_capacity -= 1

In [23]:
while (remaining_blocking_capacity>0):
    
    
    found_paths_removable = list(filter(lambda x: already_blocked_nodes[-1] not in x, found_paths_removable))
    
    if (len(found_paths_removable)<=0):
        remaining_risky=np.array(setRisky)

        for i in already_blocked_nodes:
            remaining_risky=np.delete(remaining_risky, np.where(remaining_risky[:] == i))

        if(len(remaining_risky)>0):
            if(len(remaining_risky)>remaining_blocking_capacity):
                chosen=(np.random.choice(remaining_risky,size=remaining_blocking_capacity,replace=False))  
                for items in chosen:
                    already_blocked_nodes.append(items)
            else:
                 for items in remaining_risky:
                    already_blocked_nodes.append(items)   
        
        break
    
    found_paths_removable_np = np.concatenate([np.array(i) for i in found_paths_removable])

    nodes_on_paths, total_appearance = np.unique(found_paths_removable_np, return_counts=True)

    #for i in setP:
        #total_appearance=np.delete(total_appearance, np.where(nodes_on_paths[:] == i))
        #nodes_on_paths=np.delete(nodes_on_paths, np.where(nodes_on_paths[:] == i))

    already_blocked_nodes.append(nodes_on_paths[np.argmax(total_appearance)])
    remaining_blocking_capacity -= 1


In [24]:
blocked_nodes = np.zeros(n)
#blocked_nodes[nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]-1]=1


In [25]:
blocked_nodes_df=pd.DataFrame(blocked_nodes).T
blocked_nodes_df.columns=(range(1,(n+1)))
blocked_nodes_df
blocked_nodes_df[already_blocked_nodes]=1

In [26]:
initial_solution_for_tabu=blocked_nodes_df.values.tolist()[0]


In [27]:
#blocked_nodes_df.to_csv('blocked_nodes_df-n'+str(n)+'-m'+str(m)+'.csv',index=False)


In [28]:
def simulate_LTM(removed_nodes,Graph_for_simulation,setP):
    G=Graph_for_simulation.copy()
    #silinen=[]
    for i in range(len(removed_nodes)):
        if(removed_nodes[i]>0):            
            G.remove_node(i+1)
            #silinen.append(i+1)
    converted_list=setP.copy()
    #print('Blocked:'+str(silinen)) 
    
    while(1):
        #start2=time.time()
        converted_list1 = converted_list[:]
        #print('step:'+str(converted_list))
        for node in G.nodes():
            if (node) not in converted_list: 
                total_weight = 0
                for each in G.neighbors(node):
                    if (each) in converted_list:
                        total_weight = total_weight + adjacency_df.at[node,each]
                if total_weight > adjacency_df.at[node,'Threshold']:
                    converted_list.append(node)
        #end2=time.time()
        #print(end2-start2)           
        if set(converted_list1) == set(converted_list):
            break
    objective_value=adjacency_df.loc[converted_list,'IsRisky'].sum()
    #print('FINAL:'+str(converted_list))
    return objective_value 

In [29]:
def NeighborEvaluation(Neighbor,Graph_for_simulation,setP):
    
    ObjVal = simulate_LTM(Neighbor,Graph_for_simulation,setP)
    #ObjVal = (i * Neighbor[i])
    
    return ObjVal

In [30]:
def NeighborhoodSearch(IncumbentSolution,TabuList,Graph_for_simulation,setP):
    #print('_________')
    #print('INCUMBENT')
    simulate_LTM(IncumbentSolution,Graph_for_simulation,setP)
    #print('_________')
    ZeroIndices = []
    OneIndices = []
    for i in range(len(IncumbentSolution)):
        if IncumbentSolution[i] == 0:
            ZeroIndices.append(i)
        else:
            OneIndices.append(i)
    
    AllNeighbor = []
    AllNeighborObjVals = []
    Moves = []
    random_ones= random.sample(range(len(OneIndices)),int((len(OneIndices)*ones[0])))
    random_zeros= random.sample(range(len(ZeroIndices)),int((len(ZeroIndices)*zeros[0])))
    for i in random_ones:
        for j in random_zeros:
    #for i in range(len(OneIndices)):
        #for j in range(len(ZeroIndices)):
            if ((ZeroIndices[j]+1) not in setP):
                if [OneIndices[i],ZeroIndices[j]] not in TabuList:
                    #start=time.time() 
                    Moves.append([OneIndices[i],ZeroIndices[j]])
                    Temp = copy(IncumbentSolution)
                    Temp[ZeroIndices[j]] = 1
                    Temp[OneIndices[i]] = 0
                    #print('Exchanged:'+str(OneIndices[i]+1)+','+str(ZeroIndices[j]+1))
                    TempObjVal = NeighborEvaluation(Temp,Graph_for_simulation,setP)
                    #print("neighborda çıktım")
                    AllNeighbor.append(Temp)
                    AllNeighborObjVals.append(TempObjVal)
                    #end=time.time()
                    #print(end-start)
    BestIndex = AllNeighborObjVals.index(min(AllNeighborObjVals))
    BestSolution = AllNeighbor[BestIndex]
    BestObjVal = AllNeighborObjVals[BestIndex]
    BestMove = Moves[BestIndex]

    return BestSolution, BestObjVal, BestMove

In [31]:
def TabuSearchBinarySwap(NumberOfOnes,SpinningLimit,MaxIter,TabuTenure,Graph_for_simulation,setP,initial_solution_for_tabu):
    
    random.seed(123)
    
    InitialSolution = initial_solution_for_tabu.copy() 

    IncumbentSolution = InitialSolution
    IncumbentObjVal = NeighborEvaluation(IncumbentSolution,Graph_for_simulation,setP)
    #print("tabuda çıktım")
    BestSolution = IncumbentSolution
    BestObjVal = IncumbentObjVal
    Trajectory = [IncumbentObjVal]
    BestTrajectory = [BestObjVal]
    
    TabuList = []
    #for i in setS:
     #   for j in setP:
     #       TabuList = [j,i]
    
    SpinningIndex = 0
    Iteration = 0
    while Iteration < MaxIter:
        #start=time.time()        
        Out = NeighborhoodSearch(IncumbentSolution,TabuList,Graph_for_simulation,setP)
        #end=time.time()
        #print(end-start)
        BestNeighbor = Out[0]
        BestNeighborObjVal = Out[1]
        BestNeighborMove = Out[2]
        
        if BestObjVal <= IncumbentObjVal:
            
            IncumbentSolution = BestNeighbor
            IncumbentObjVal = BestNeighborObjVal
            Trajectory.append(IncumbentObjVal)
            if IncumbentObjVal < BestObjVal:
                BestObjVal = IncumbentObjVal
                BestSolution = IncumbentSolution
                SpinningIndex = 0
            else:
                SpinningIndex += 1
            BestTrajectory.append(BestObjVal)
            TabuList.append(BestNeighborMove)
            
        else:
            
            Trajectory.append(IncumbentObjVal)
            BestTrajectory.append(BestObjVal)
            SpinningIndex += 1
            
            
        if len(TabuList) > TabuTenure:
            TabuList.pop(0)
            
        if SpinningIndex > SpinningLimit:
            
            Iteration = MaxIter
            
        else:
        
            Iteration += 1
    
    return BestSolution, BestObjVal, BestTrajectory, Trajectory, TabuList



In [32]:

SpinningLimit = 5
MaxIter = n*5
TabuTenure = 20
Graph2 = convert(adjacency_df)
start1=time.time()
Tabu_Results = TabuSearchBinarySwap(Blocking_Capacity,SpinningLimit,MaxIter,TabuTenure,Graph2,setP,initial_solution_for_tabu)
end1=time.time()
print(end1-start1)

1282.535549879074


In [33]:
Tabu_Results[1]

25.0

In [34]:
utku=[]
for i in range(len(Tabu_Results[0])):
    if Tabu_Results[0][i]==1: 
        utku.append(i+1)
print(utku)

[14, 28, 29, 46, 47, 49, 55, 57, 61, 65, 68, 70, 74, 90, 96, 97, 101, 103, 108, 111, 119, 144, 147, 149, 150, 158, 161, 162, 170, 175, 180, 183, 185, 192, 194, 202, 203, 204, 211, 218, 219, 221, 224, 235, 243, 246, 253, 254, 255, 256, 257, 267, 269, 271, 277, 279, 280, 283, 289, 295]


In [35]:
[12, 14, 18, 23, 25, 29, 32, 34, 40, 41, 47, 52, 73, 85, 86, 87, 89, 90, 91, 97]

[12,
 14,
 18,
 23,
 25,
 29,
 32,
 34,
 40,
 41,
 47,
 52,
 73,
 85,
 86,
 87,
 89,
 90,
 91,
 97]

In [36]:
NeighborEvaluation(initial_solution_for_tabu,Graph,setP)

29.0

In [37]:
bosliste=np.zeros(100).tolist()
NeighborEvaluation(bosliste,Graph,setP)

89.0